In [16]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import keras
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Activation, Flatten, Add
from keras.layers import BatchNormalization
# from keras.layers import GlobalAveragePooling2D
# from keras.optimizers import Adam, SGD
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D

from keras import backend
import matplotlib.pyplot as plt


In [8]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
xx_train = xx_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(xx_train, axis=0)
xx_train -= mean
x_test -= mean

x_train = xx_train[:45000]
y_train = yy_train[:45000]
x_valid = xx_train[45000:50000]
y_valid = yy_train[45000:50000]

x_train = x_train/255.0
x_valid = x_valid/255.0
x_test = x_test/255.0

y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
y_test = np_utils.to_categorical(y_test)

In [3]:
x_train.shape[1:]

(32, 32, 3)

In [4]:
def add(a,b):
    shape1 = backend.int_shape(a)
    shape2 = backend.int_shape(b)
    w = int(round(shape1[1]/shape2[1]))
    h = int(round(shape1[2]/shape2[2]))
    eq = shape1[3] == shape2[3]
    
    tmp = a
    print w,h,eq
    print shape1, shape2
    if w>1 or h>1 or not eq:
        tmp = Conv2D(filters=shape2[3],kernel_size=(1,1),strides=(w,h),padding='valid',kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(a)
    print backend.int_shape(tmp)
    print
    return Add()([tmp, b])

### Modified ResNet v2

In [5]:
def layer(num_filt, size, strides, inp):
    tmp = BatchNormalization(axis=3)(inp)
    tmp = Activation('relu')(tmp)
    tmp = Conv2D(num_filt, size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(tmp)
    return tmp

l = Input(x_train.shape[1:])

x = Conv2D(128, (7,7), strides=2, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(l)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)

#first layer
num = 32
xtmp = Conv2D(num, (1,1), strides=1, 
              padding='same', 
              kernel_initializer='he_normal', 
              kernel_regularizer=l2(0.0001))(x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)

x = add(x,xtmp)

#other layer
xtmp = layer(num, (1,1), 1, x)
xtmp = layer(num, (3,3), 1, xtmp)
xtmp = layer(num*4, (1,1), 1, xtmp)
x = add(x,xtmp)

for i in xrange(9):
    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

    #other layer
    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

x = BatchNormalization()(x)
x = Activation('relu')(x)

xshape = backend.int_shape(x)
x = AveragePooling2D(pool_size=(xshape[1],xshape[2]), strides=(1,1))(x)
x = Flatten()(x)

x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None, 16, 16, 128) (None, 16, 16, 128)
(None, 16, 16, 128)

1 1 True
(None

In [6]:
model = Model(l,x)

model.compile(
    optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 128)  18944       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 128)  512         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 128)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
#if training was cut short
model = load_model('weights.best.hdf5')

In [9]:
history = model.fit(x_train,y_train,
          batch_size=32, epochs=200,
          shuffle=True,
          validation_data=(x_valid, y_valid), 
          callbacks=[checkpoint])

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 1545s 34ms/step - loss: 0.7966 - acc: 0.8058 - val_loss: 1.1426 - val_acc: 0.7316
Epoch 2/200
45000/45000 [==============================] - 1517s 34ms/step - loss: 0.7692 - acc: 0.8158 - val_loss: 0.9081 - val_acc: 0.7712
Epoch 3/200
45000/45000 [==============================] - 1514s 34ms/step - loss: 0.7405 - acc: 0.8244 - val_loss: 0.9733 - val_acc: 0.7486
Epoch 4/200
45000/45000 [==============================] - 1497s 33ms/step - loss: 0.7213 - acc: 0.8326 - val_loss: 0.9870 - val_acc: 0.7494
Epoch 5/200
45000/45000 [==============================] - 1500s 33ms/step - loss: 0.6991 - acc: 0.8407 - val_loss: 1.0767 - val_acc: 0.7344
Epoch 6/200
45000/45000 [==============================] - 1506s 33ms/step - loss: 0.6851 - acc: 0.8477 - val_loss: 0.9288 - val_acc: 0.7664
Epoch 7/200
45000/45000 [==============================] - 1508s 34ms/step - loss: 0.6659 - acc: 0.8562 -

KeyboardInterrupt: 

In [10]:
res = model.evaluate(x_test, y_test, batch_size=512)

10000/10000 [==============================] - 185s 19ms/step


### Result after stopping at 60 epochs

In [11]:
print('Loss: %.2f Accuracy: %.2f%%' % (res[0], res[1]*100.0))

Loss: 1.10 Accuracy: 78.19%


### Best result of 60 epochs

In [12]:
model2 = load_model('weights.best.hdf5')
res = model2.evaluate(x_test, y_test, batch_size=512)
print('Loss: %.2f Accuracy: %.2f%%' % (res[0], res[1]*100.0))

10000/10000 [==============================] - 142s 14ms/step
Loss: 0.98 Accuracy: 80.06%


In [17]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()